In [6]:
import pickle, os, string, re, copy
from nltk.stem import PorterStemmer

#This is the class definition
class Posting_List(object):
    def __init__(self):
#Here we are initializing the class instance variables, total_count is set to 0,
#token is an empty string and occurrance is an empty dictionary
        self.total_count = 0
        self.token = ''
        self.occurrance = {
        }
#This is the representation of the instance when it is printed 
    def __repr__(self):
#Here we are getting the keys of the occurrance dictionary 
        dict = self.occurrance.keys()
#We are casting the keys to a list object so that we can iterate over them
        lister = list(dict)
#We are checking if the list is not empty, if it is not empty then we are returning the list of doc_ids
        if lister:
            return f'Result-set: {lister}'
#if the list is empty we are returning an empty result set
        else:
            return 'Result_set:   '
#This is to get the length of the occurrance dictionary
    def __len__(self):
        return len(self.occurrance)
#The appendoccurence method adds the doc_id and position to the occurrance dictionary
    def appendoccurence(self, doc_id, position):
#Here we are incrementing the total count
        self.total_count += 1
#Here we are checking if the doc_id is not in the occurrance dictionary, if it is not present we are adding it to the dictionary
        if doc_id not in self.occurrance.keys():
            self.occurrance[doc_id] = []
#Here we are appending the position to the doc_id in the occurrance dictionary
        self.occurrance[doc_id].append(position)
    
class InvertedIndex(object):
    # Constructor to initialize the index and document dictionaries
    def __init__(self):
        self.index = {}     # dictionary to store the inverted index
        self.docs = {}      # dictionary to store the documents

    # Method to get the posting list of a given term
    def get_term_postings(self, term):
        if term in self.index.keys():   # if the term is present in the index
            return self.index[term]    # return its posting list
        else:
            return f'{term} is not present'+ Posting_List()   # otherwise, return a message indicating that the term is not present
    
    # Method to get the length of the index
    def __len__(self):
        return len(self.index.keys())


# Remove Punctuation
def remove_punctuation(word):
    return word.translate(word.maketrans('','',string.punctuation))

vocab = set()
doc_contents = []
Invert_Index_Main = InvertedIndex()
printable = set(string.printable) 
# Printable characters are
# 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
# !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c

# Initialize the stemmer and stop words set
query_parser = PorterStemmer()
stop_words = set()
# Load stop words from file
with open('Stopword-List.txt', 'r') as stopwordlist:
    lines = stopwordlist.readlines()
    for line in lines:
        stop_word = line.strip()
        if stop_word:
            stop_words.add(stop_word)

# Loop through all files
for file_number in range(1, 31):
    # Open the file and read lines
    with open(f'{file_number}.txt', 'r') as file1:
        lines = file1.readlines()
        # Initialize position and document set for this file
        position = {'doc': file_number, 'row': 0, 'col': 0, 'token_no': 0}
        ddocuments = set()
        
        # Loop through each line in the file
        for line_no, line in enumerate(lines):
            # Update position
            position['row'] = line_no 
            position['col'] = 0
            
            # Split the line into words
            words = re.split('[.\s,?!:;-]', line)
            
            # Loop through each word in the line
            for word in words:
                # Update position and token number
                position['col'] += len(word) + 1
                position['token_no'] += 1
                
                # Preprocess the word
                word = word.lower() # Case folding
                word = ''.join(filter(lambda x: x in printable, word)) # Filter non-ASCII characters
                word = remove_punctuation(word) # Remove punctuations
                
                if re.match('\d+[A-Za-z]+', word):
                    word = re.split('\d+', word)[1]
                if re.match('[A-Za-z]+\d+', word):
                    word = re.split('\d+', word)[0]
                
                # Skip the word if it is a stop word or empty
                if len(word) <= 1 or word in stop_words:
                    continue
                
                # Stem the word
                word = query_parser.stem(word)
                
                # Add the word to vocabulary and document set
                vocab.add(word)               
                ddocuments.add(word)
                
                # Update the inverted index
                if word in Invert_Index_Main.index:
                    Invert_Index_Main.index[word].appendoccurence(file_number, copy.deepcopy(position)) 
                else:
                    plist = Posting_List()
                    Invert_Index_Main.index[word] = plist
                    Invert_Index_Main.index[word].appendoccurence(file_number, copy.deepcopy(position))
                    
        # Add the document set to the inverted index
        Invert_Index_Main.docs[file_number] = ddocuments
        doc_contents.append(ddocuments)
# Clean Query Term
def improvise_text(word):
    # Case Folding
    word = word.lower()
    # Filter non-alphanumeric characters
    word = ''.join(filter(lambda x: x.isalnum(), word))
    # Remove Punctuations, digits and split numbers from alpha-numeric strings
    if not (word == '(' or word == ')'):
        word = re.split('\d+', word)[0] or re.split('\D+', word)[-1]
    # Skip stemming if already in stemmed form
    if not word.endswith('ly') and not word.endswith('ed') and not word.endswith('ing'):
        word = query_parser.stem(word)
    return word

# To Save Inverted Index to File
with open('Invert_Index_Main.p', 'wb') as index_file:
    pickle.dump(Invert_Index_Main, index_file)

# To Load Inverted Index from File
try:
    with open('Invert_Index_Main.p', 'rb') as index_file:
        Invert_Index_Main = pickle.load(index_file)
except FileNotFoundError:
    print("Inverted Index file not found!")

# Function to compute the intersection of two posting lists
def intersection(p1, p2):
    if len(p1) == 0 or len(p2) == 0:   # if one of the posting lists is empty
        return Posting_List()         # return an empty posting list

    if isinstance(p1, set) and isinstance(p2, set):   # if both posting lists are sets
        return p1.intersection(p2)    # compute the intersection using the set intersection method
    elif isinstance(p1, set):         # if only p1 is a set
        return p1.intersection(p2.occurrance.keys())  # compute the intersection using p1 as a set and p2's keys
    elif isinstance(p2, set):         # if only p2 is a set
        return p2.intersection(p1.occurrance.keys())  # compute the intersection using p2 as a set and p1's keys

    # If both posting lists are not sets, create a new posting list
    pn = Posting_List()
    for pn_keys in (p1.occurrance.keys() & p2.occurrance.keys()):  # iterate over the common keys of p1 and p2
        pn.appendoccurence(pn_keys, p1.occurrance[pn_keys])  # add the occurrences of the key in p1
        pn.appendoccurence(pn_keys, p2.occurrance[pn_keys])  # add the occurrences of the key in p2
    return pn


def union(p1, p2):
    if len(p1) == 0:
        return p2
    elif len(p2) == 0:
        return p1
    if isinstance(p1, set) and isinstance(p2, set):
        return p1.union(p2)
    elif isinstance(p1, set):
        return p1.union(p2.occurrance.keys())
    elif isinstance(p2, set):
        return p2.union(p1.occurance.keys())
    
    pn = Posting_List()
    for pn1_keys in p1.occurrance.keys() :
        pn.appendoccurence(pn1_keys, p1.occurrance[pn1_keys])
    for pn2_keys in p2.occurrance.keys() :
        pn.appendoccurence(pn2_keys, p2.occurrance[pn2_keys])
    
    return pn

# Function to find the documents that do not contain a given query term
def inverted_post(Invert_Index_Main, p):
    print(p)  # print the query term or set of terms
    if isinstance(p, set):  # if p is a set of terms
        print('Returning ')
        print(set(Invert_Index_Main.docs).difference(p))
        return set(Invert_Index_Main.docs).difference(p)  # return the set of all documents minus the query terms
    else:  # if p is a single query term
        print(set(Invert_Index_Main.docs).difference(set(p.occurrance.keys())))
        return set(Invert_Index_Main.docs).difference(set(p.occurrance.keys()))  # return the set of all documents minus the documents containing the query term
def construct_indices():
    # Set path to data directory
    path_to_data = os.path.dirname(__file__) + '../../data/'

    # Initialize empty vocabulary set and document contents list
    vocab = set()
    doc_contents = []

    # Create a new instance of the InvertedIndex class
    Invert_Index_Main = InvertedIndex()

    # Set of printable characters
    printable = set(string.printable)

    # Initialize a PorterStemmer object for stemming words
    query_parser = PorterStemmer()

    # Initialize a set of stop words
    stop_words = set()

    # Load stop words from file
    with open(path_to_data+'Stopword-List.txt', 'r') as stopwordlist:
        lines = stopwordlist.readlines()
        for line in lines:
            stop_words.add(line.split('\n')[0])
        stop_words.remove('')

    # Loop over all files in the data directory
    for file_number in range(1, 31):
        # Open file
        with open(path_to_data + f'{file_number}.txt', 'r') as file1:
            # Read lines from file
            lines = file1.readlines()

            # Print file number and first line of file
            print(f'File Number : {file_number}.txt' )
            print(lines[0])

            # Initialize position dictionary for keeping track of token positions
            position = {'doc':file_number,'row':0, 'col':0, 'token_no':0}

            # Loop over lines in file
            for line_no,line in enumerate(lines):
                # Initialize empty set for current document's words
                ddocuments = set()

                # Split line into words at . , whitespace ? ! : ;
                position['row'] = line_no 
                position['col'] = 0
                for word in re.split('[.\s,?!:;-]', line):
                    # Update position
                    position['col'] += len(word) + 1
                    position['token_no'] += 1

                    # Case folding
                    word = word.lower()
                    
                    # Filter non-ASCII characters
                    word = ''.join(filter(lambda x: x in printable, word))
                    
                    # Remove punctuations
                    word = remove_punctuation(word)

                    # Remove digits attached to alphabets
                    if re.match('\d+[A-Za-z]+',word):
                        word = re.split('\d+',word)[1]
                    # Remove alphabets attached to digits
                    if re.match('[A-Za-z]+\d+',word):
                        word = re.split('\d+',word)[0]
                    
                    # Skip word if it is a stop word, empty or has only one character
                    if len(word) == 0 or len(word) == 1 or word == '' or word == ' ':
                        continue
                    if word in stop_words:
                        continue

                    # Stem word using PorterStemmer
                    word = query_parser.stem(word)                        
                    vocab.add(word)                    
                    ddocuments.add(word)

                    # Add word to inverted index
                    if word in Invert_Index_Main.index.keys():
                        Invert_Index_Main.index[word].appendoccurence(file_number, copy.deepcopy(position)) 
                    else:
                        plist = Posting_List()
                        Invert_Index_Main.index[word] = plist
                        Invert_Index_Main.index[word].appendoccurence(file_number, copy.deepcopy(position))
                        
            Invert_Index_Main.docs[file_number] = ddocuments
            doc_contents.append(ddocuments)
    ii = InvertedIndex()
    ii.status = True
    ii.data = Invert_Index_Main
    ii.save()
    return True


binary = ['and', 'or']
unary = ['not']

# Token types
LPAREN, RPAREN, EOF, TERM, AND, OR, NOT = (
    '(', ')', 'EOF', 'TERM', 'AND','OR', 'NOT'
)


class Token(object):
    def __init__(self, type, value):
        self.type = type
        self.value = value
        self.inverse = False
        self.row = []

    def __str__(self):
        #String representation of the class instance.

        return 'Token({type}, {value})'.format(
            type=self.type,
            value=repr(self.value)
        )

    def __repr__(self):
        return self.__str__()


class Lexer(object):
    def __init__(self, text):
        self.text = text
        self.pos = 0
        self.current_char = self.text[self.pos]
        

    def error(self):
        raise Exception('Invalid character')

    def advance(self):
        self.pos += 1
        if self.pos > len(self.text) - 1:
            self.current_char = None  # Indicates end of input
        else:
            self.current_char = self.text[self.pos]

    def skip_whitespace(self):
        while self.current_char is not None and self.current_char.isspace():
            self.advance()

    def integer(self):
        result = ''
        while self.current_char is not None and self.current_char.isdigit():
            result += self.current_char
            self.advance()
        return int(result)
    
    def word(self):
        result = ''
        # while self.current_char is not None and (self.current_char.isalpha() or self.current_char == '_'):
        while self.current_char is not None and (self.current_char in printable) and (self.current_char not in (' ', '|','&','!', '(', ')')):
            result += self.current_char
            self.advance()
        return str(result)

    def get_next_token(self):
    # Mapping between characters and tokens
        char_to_token = {
            '&': Token(AND, 'AND'),
            '|': Token(OR, 'OR'),
            '!': Token(NOT, 'NOT'),
            '(': Token(LPAREN, '('),
            ')': Token(RPAREN, ')')
        }

    # Buffer for storing a word
        wbuff = ''

        while self.current_char is not None:
            if self.current_char.isspace():
                self.skip_whitespace()
                continue

            if self.current_char in char_to_token:
                token = char_to_token[self.current_char]
                self.advance()
                return token

            if self.current_char.isalpha():
            # Use a buffer to store the characters of a word
                wbuff = self.current_char
                self.advance()
                while self.current_char is not None and self.current_char.isalpha():
                    wbuff += self.current_char
                    self.advance()
                return Token(TERM, wbuff)

            self.error()

        return Token(EOF, None)

class AST(object):
    pass


class Operat(AST):
    def __init__(self, left, op, right):
        self.left = left
        self.token = self.op = op
        self.right = right
        self.inverse = False
        self.row = []
        self.value = ''
        
class Num(AST):
    def __init__(self, token):
        self.token = token
        self.value = token.value
        self.inverse = False
        self.row = []

class Parser(object):
    def __init__(self, lexer):
        self.lexer = lexer
        # set current token to the first token taken from the input
        self.current_token = self.lexer.get_next_token()

    def error(self):
        raise Exception('Invalid syntax')

    def eat(self, token_type):
        # compare the current token type with the passed token
        # type and if they match then "eat" the current token
        # and assign the next token to the self.current_token,
        # otherwise raise an exception.
        if self.current_token.type == token_type:
            self.current_token = self.lexer.get_next_token()
        else:
            self.error()

    def factor(self):
        """factor : INTEGER | LPAREN expr RPAREN"""
        token = self.current_token
        
        
        if token.type == TERM:
            self.eat(TERM)
            return Num(token)
        
        elif token.type == NOT:
            self.eat(NOT)
            node = self.expr()
            node.inverse = True
            return node
            
        
        
        elif token.type == LPAREN:
            self.eat(LPAREN)
            node = self.expr()
            self.eat(RPAREN)
            return node

    def term(self):
        node = self.factor()
        while self.current_token.type in (AND,):
            token = self.current_token
         
            if token.type == AND:
                self.eat(AND)

            node = Operat(left=node, op=token, right=self.factor())

        return node

    def expr(self):
        node = self.term()
        while self.current_token.type in (OR,):
            token = self.current_token
            if token.type == OR:
                self.eat(OR)
            
            node = Operat(left=node, op=token, right=self.term())
        
        return node

    def parse(self):
        return self.expr()


class NodeVisitor(object):
    def visit(self, node):
        method_name = 'visit_' + type(node).__name__
        visitor = getattr(self, method_name, self.generic_visit)
        return visitor(node)

    def generic_visit(self, node):
        raise Exception('No visit_{} method'.format(type(node).__name__))


class Interpreter(NodeVisitor):
    def __init__(self, parser, index, query_parser):
        # Initialize the Interpreter with a Parser, an Index, and a Query Parser
        self.parser = parser
        self.index = index
        self.query_parser = query_parser

    def visit_Operat(self, node):
        # Visit the left and right children of the current node
        left = self.visit(node.left)   
        right = self.visit(node.right)
    
        # If the operator is an AND operator
        if node.op.type == AND:
            # If the left node is inverted, add a negation symbol to its value and set its inverse flag to False
            if left.inverse:
                left.value = '!' + str(left.value)
                left.inverse = False
            # If the right node is inverted, add a negation symbol to its value and set its inverse flag to False
            if right.inverse:
                right.value = '!' + str(right.value)
                right.inverse = False
            # Compute the intersection of the posting lists of the left and right nodes
            node.row = intersection(left.row, right.row)
        # If the operator is an OR operator
        elif node.op.type == OR:
            # If the left node is inverted, add a negation symbol to its value and set its inverse flag to False
            if left.inverse:
                left.value = '!' + str(left.value)          
                left.inverse = False
            # If the right node is inverted, add a negation symbol to its value and set its inverse flag to False
            if right.inverse:
                right.value = '!' + str(right.value)            
                right.inverse = False
            # Compute the union of the posting lists of the left and right nodes
            node.row = union(left.row, right.row)
    
        # If the current node is inverted
        if node.inverse:
            # Compute the complement of its posting list
            node.row = inverted_post(self.index, node.row)
            # Set its inverse flag to False
            node.inverse = False
    
        # Return the current node
        return node
       
    def visit_Num(self, node):
        node.value = node.value.split('_')[0]
        if self.query_parser.stem(node.value) in self.index.index.keys():
            term_docs = self.index.index[self.query_parser.stem(node.value)]
        else:
            term_docs = {}

        node.row = term_docs
        if node.inverse == True:
            node.row = inverted_post(self.index, node.row)
            node.inverse = False
        return node

    def interpret(self):
        tree = self.parser.parse()
        return self.visit(tree)

'''
This function takes a query in natural language as an input and converts it into a boolean query.
It uses the PorterStemmer and Lexer objects from nltk library to tokenize the natural language query and identify the words.
The Parser class from nltk is used to parse the tokens using the given grammar.
The Interpreter class from nltk is used to interpret the parsed tokens and convert the natural language query into a boolean query.
The boolean query is then returned as the output.
'''
def get_boolean_query(query):
    text = str(query)
    text = text.replace(' and ','&')
    text = text.replace(' AND ','&')
    text = text.replace(' or ','|')
    text = text.replace(' OR ','|')
    text = text.replace('NOT', '!')
    text = text.replace('not ','!')

    query_parser = PorterStemmer()
    lexer = Lexer(text)
    parser = Parser(lexer)
    interpreter = Interpreter(parser, Invert_Index_Main, query_parser)
    result = interpreter.interpret()
    return result.row

def positional_intersect(p1, p2, k):
    
    ip = intersection(p1, p2)
    lip = sorted(list(ip.occurrance))
    npl = Posting_List()
    ans = []
    
    for doc in lip:
        positions1 = p1.occurrance[doc]
        positions2 = p2.occurrance[doc]
        for pos1 in positions1:
            for pos2 in positions2:
                if pos2['token_no'] -  pos1['token_no'] == k and pos2['token_no'] -  pos1['token_no'] > 0:
                    ans.append({'doc':doc, 'pos1':  pos1, 'pos2':pos2})
                    npl.appendoccurence(doc,pos1)
                    npl.appendoccurence(doc,pos2)
    return npl
        

def get_phrasal_query(query):
    text = str(query)
    try:
        q1, q2 = text.split(' ')
    except ValueError as e:
        raise ValueError('Invalid Phrasal Query Syntax')
    query_parser = PorterStemmer()
    q1 = query_parser.stem(q1)
    q2 = query_parser.stem(q2)
    result = [] 
    p1 = Invert_Index_Main.get_term_postings(q1)
    p2 = Invert_Index_Main.get_term_postings(q2)
    
    result = positional_intersect(p1, p2, 1)
    
    return result

def get_proximity_query(query):
    text = str(query)
    try:
        q1, q2, q3 = text.split(' ')
    except ValueError as e:
        raise ValueError('Invalid Proximity Query Syntax')
    query_parser = PorterStemmer()
    q1 = query_parser.stem(q1)
    q2 = query_parser.stem(q2)
    k = int(q3[1])+ 1
    print('Inverted Index')
    result = [] 
    
    p1 = Invert_Index_Main.get_term_postings(q1)
    p2 = Invert_Index_Main.get_term_postings(q2)
    print(p1)
    result = positional_intersect(p1, p2, k-1)

    return result

#GUI Implementation using Tkinter modules
import tkinter as tk

def open_boolean_query_window():
    boolean_query_window = tk.Toplevel(root)
    boolean_query_window.title("Boolean Query")

    query_label = tk.Label(boolean_query_window, text="Enter Boolean Query: (batter and bowler) or (batter or bowler)", font='Arial 10 bold')
    query_label.pack()

    query_entry = tk.Entry(boolean_query_window, width=50)
    query_entry.pack()

    def get_query():
        q = query_entry.get()
        result = get_boolean_query(q)
        output_box.delete(1.0, tk.END) # clear previous output
        output_box.insert(tk.END, result)

    submit_button = tk.Button(boolean_query_window, text="Submit", command=get_query, width=20, height=2, font='Arial 10 bold')
    submit_button.pack()

    output_label = tk.Label(boolean_query_window, text="Result:", font='Arial 10 bold')
    output_label.pack()

    output_box = tk.Text(boolean_query_window, height=10, width=50)
    output_box.pack()

def open_phrasal_query_window():
    phrasal_query_window = tk.Toplevel(root)
    phrasal_query_window.title("Phrasal Query")

    query_label = tk.Label(phrasal_query_window, text="Enter phrasal Query: [Babar Azam]", font='Arial 10 bold')
    query_label.pack()

    query_entry = tk.Entry(phrasal_query_window, width=50)
    query_entry.pack()

    def get_query():
        q = query_entry.get()
        result = get_phrasal_query(q)
        output_box.delete(1.0, tk.END) # clear previous output
        output_box.insert(tk.END, result)

    submit_button = tk.Button(phrasal_query_window, text="Submit", command=get_query, width=20, height=2, font='Arial 10 bold')
    submit_button.pack()

    output_label = tk.Label(phrasal_query_window, text="Result:", font='Arial 10 bold')
    output_label.pack()

    output_box = tk.Text(phrasal_query_window, height=10, width=50)
    output_box.pack()

def open_proximity_query_window():
    proximity_query_window = tk.Toplevel(root)
    proximity_query_window.title("Proximity Query")

    query_label = tk.Label(proximity_query_window, text='Enter Proximity Query: [fielder batter /2] : ', font='Arial 10 bold')
    query_label.pack()

    query_entry = tk.Entry(proximity_query_window, width=50)
    query_entry.pack()

    def get_query():
        q = query_entry.get()
        result = get_proximity_query(q)
        output_box.delete(1.0, tk.END) # clear previous output
        output_box.insert(tk.END, result)

    submit_button = tk.Button(proximity_query_window, text="Submit", command=get_query, width=20, height=2, font='Arial 10 bold')
    submit_button.pack()

    output_label = tk.Label(proximity_query_window, text="Result:", font='Arial 10 bold')
    output_label.pack()

    output_box = tk.Text(proximity_query_window, height=10, width=50)
    output_box.pack()

def Info():
    import tkinter as tk
    root = tk.Tk()
    root.title('Inverted Index Stats')
    root.geometry('300x100')
    vocabulary_size = len(Invert_Index_Main.index.keys())
    num_documents = len(Invert_Index_Main.docs)
    message = f'Total Vocabulary Size: {vocabulary_size}\nTotal Number of Documents: {num_documents}'
    label = tk.Label(root, text=message)
    label.pack(pady=20)
    root.after(10000, lambda: root.destroy())


def exit_program():
    root.destroy()

root = tk.Tk()
root.title("Information Retrieval Assignment 1")
root.geometry("500x350")

option1_button = tk.Button(root, text="Boolean Query", command=open_boolean_query_window, width=20, height=2, font='Arial 10 bold')
option1_button.pack()

option2_button = tk.Button(root, text="Phrasal Query", command=open_phrasal_query_window, width=20, height=2, font='Arial 10 bold')
option2_button.pack()

option3_button = tk.Button(root, text="Proximity Query", command=open_proximity_query_window, width=20, height=2, font='Arial 10 bold')
option3_button.pack()

option4_button = tk.Button(root, text="Dataset Information", command=Info, width=20, height=2, font='Arial 10 bold')
option4_button.pack()

exit_button = tk.Button(root, text="Exit", command=exit_program, width=20, height=2, font='Arial 10 bold')
exit_button.pack()

root.mainloop()

Inverted Index
Result-set: [1, 6, 8, 11, 12, 14, 25, 27, 28, 29]
